In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/twitter-entity-sentiment-analysis/twitter_validation.csv
/kaggle/input/twitter-entity-sentiment-analysis/twitter_training.csv


In [2]:
def get_label(arg):
    if arg=='Positive':
        return 0
    elif arg=='Neutral':
        return 1
    elif arg=='Negative':
        return 2

In [3]:
import pandas as pd
from datasets import Dataset, DatasetDict

# Load your CSV dataset
df = pd.read_csv("/kaggle/input/twitter-entity-sentiment-analysis/twitter_training.csv")
df.columns = ['id','border','label','text']
df = df[['text','label']]
df = df.dropna().reset_index(drop=True)
df = df[(df.label=='Positive')|(df.label=='Neutral')|(df.label=='Negative')]
df['label'] = df.label.apply(lambda x: get_label(x))
df['label'] = df['label'].astype(int)
df.head()

,text,label
0,I am coming to the borders and I will kill you...,0
1,im getting on borderlands and i will kill you ...,0
2,im coming on borderlands and i will murder you...,0
3,im getting on borderlands 2 and i will murder ...,0
4,im getting into borderlands and i can murder y...,0


In [4]:
import math
df_test = df[(math.floor(df.shape[0]*0.7)):]
df = df[0:(math.floor(df.shape[0]*0.7))]

print('train:', format(df.shape[0],','))
print('test:', format(df_test.shape[0],','))

train: 42,784
test: 18,336


In [5]:
df.label.unique()

array([0, 1, 2])

In [6]:
import torch
from torch import optim

print(torch.cuda.is_available())
print(torch.cuda.current_device())

True
0


In [7]:
from torch.nn.utils.rnn import pad_sequence

class MyDataset(Dataset):
    def __init__(self, texts, labels, tokenizer):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]

        # Tokenize input text
        tokenized_input = self.tokenizer(text, padding=True, truncation=True, return_tensors='pt')

        # Return a dictionary with the tokenized input and label
        return {
            'input_ids': tokenized_input['input_ids'].squeeze(),
            'attention_mask': tokenized_input['attention_mask'].squeeze(),
            'token_type_ids': tokenized_input['token_type_ids'].squeeze(),
            'labels': torch.tensor(label)
        }

# Example usage
from torch.utils.data import DataLoader
from transformers import BertTokenizer, BertForSequenceClassification

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)
# Move the model to the GPU
model.to('cuda')

# Create an instance of the custom dataset
texts = df['text'].tolist()
labels = df['label'].tolist()
my_dataset = MyDataset(texts, labels, tokenizer)

def my_collate_fn(batch):
    input_ids = pad_sequence([item['input_ids'] for item in batch], batch_first=True)
    attention_mask = pad_sequence([item['attention_mask'] for item in batch], batch_first=True)
    token_type_ids = pad_sequence([item['token_type_ids'] for item in batch], batch_first=True)
    labels = torch.stack([item['labels'] for item in batch])
    
    return {
        'input_ids': input_ids.to('cuda'),
        'attention_mask': attention_mask.to('cuda'),
        'token_type_ids': token_type_ids.to('cuda'),
        'labels': torch.tensor(labels).to('cuda')
    }

# Initialize optimizer (you need to define it somewhere in your code)
optimizer = optim.AdamW(model.parameters(), lr=2e-5)
# Create a DataLoader using the custom dataset with collate_fn argument
train_data_loader = DataLoader(my_dataset, batch_size=32, shuffle=True, collate_fn=my_collate_fn)

# Example fine-tuning loop (simplified)
num_epochs = 3
for epoch in range(num_epochs):
    for batch in train_data_loader:
        inputs = {
            'input_ids': batch['input_ids'],
            'attention_mask': batch['attention_mask'],
            'token_type_ids': batch['token_type_ids'],
            'labels': batch['labels']
        }
        # Move the data to the GPU
        for key in inputs:
            inputs[key] = inputs[key].to('cuda')
        optimizer.zero_grad()
        outputs = model(**inputs)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_34/2472744794.py:51: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'labels': torch.tensor(labels).to('cuda')


In [8]:
def get_label(arg):
    if arg==0:
        return 'Positive'
    elif arg==1:
        return 'Neutral'
    elif arg==2:
        return 'Negative'

In [11]:
def get_pred(arg):
    tokenized_inputs = tokenizer(arg, padding=True, truncation=True, return_tensors='pt').to('cuda')

    with torch.no_grad():
        outputs = model(**tokenized_inputs)

    logits = outputs.logits

    probabilities = torch.nn.functional.softmax(logits, dim=-1)
    predicted_class = torch.argmax(probabilities, dim=-1).item()

    #predicted_labels = get_label(predicted_class)
    predicted_labels = predicted_class
    return predicted_labels

In [17]:
from tqdm.notebook import tqdm
tqdm.pandas()

In [18]:
df['pred_label'] = df['text'].progress_apply(lambda x: get_pred(x))
df.head()

  0%|          | 0/42784 [00:00<?, ?it/s]

,text,label,pred_label
0,I am coming to the borders and I will kill you...,0,0
1,im getting on borderlands and i will kill you ...,0,0
2,im coming on borderlands and i will murder you...,0,0
3,im getting on borderlands 2 and i will murder ...,0,0
4,im getting into borderlands and i can murder y...,0,0


In [19]:
correct_predictions = (df['label'] == df['pred_label']).sum()
total_predictions = len(df)
accuracy = correct_predictions / total_predictions
print(f"Accuracy: {accuracy:.2%}")

Accuracy: 97.99%


In [20]:
df_test['pred_label'] = df_test.text.progress_apply(lambda x: get_pred(x))
df_test.head()

  0%|          | 0/18336 [00:00<?, ?it/s]

,text,label,pred_label
52493,@ RockstarGames does not work red dead redeemm...,2,2
52494,@RockstarGames not working red dead redemption...,2,2
52495,@RockstarGames not on working red dead redempt...,2,2
52496,@RockstarGames not complete red dead redemptio...,2,2
52497,Skinned this in red dead redemption 2 yesterda...,1,0


In [21]:
correct_predictions = (df_test['label'] == df_test['pred_label']).sum()
total_predictions = len(df_test)
accuracy = correct_predictions / total_predictions
print(f"Accuracy: {accuracy:.2%}")

Accuracy: 66.55%
